### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

# Importando as bibliotecas

In [ ]:
import os
from llama_index.llms.oci_genai import OCIGenAI
from llama_index.core.agent import ReActAgent #Framework to work with agents - they use a lot of intermediate prompts
from llama_index.core.tools import FunctionTool
from llama_index.core.llms import ChatMessage

# Acessando suas credenciais

In [2]:
compartment_id = os.environ["NB_SESSION_COMPARTMENT_OCID"]

# Ferramentas que o agente irá utilizar

In [6]:
def add(inputs: str) -> int:
    """Add two numbers given as a comma-separated string."""
    a, b = map(int, inputs.split(","))
    return a + b

def multiply(inputs: str) -> int:
    """Multiply two numbers given as a comma-separated string."""
    a, b = map(int, inputs.split(","))
    return a * b

def square(inputs: str) -> int:
    """Calculates the square of a number."""
    a = int(inputs)
    return a * a

# Define the tools
tools = [add, multiply, square]

functions = []
for tool in tools:
    #FunctionTool integrate the functions as tools for the agent
    functions.append(FunctionTool.from_defaults(tool)) 

# Criando o objeto do modelo parametrizado

In [7]:
command_r_plus = "ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceya7ozidbukxwtun4ocm4ngco2jukoaht5mygpgr6gq2lgq"
# Initialize LLM with OCI Generative AI
llm = OCIGenAI(
    model=command_r_plus,
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id=compartment_id,
    provider="cohere",
    context_size="128000",
    temperature=0,
    #max_tokens=2500
)

# Criando o objeto do AI Agent

In [7]:
# Initialize ReAct agent with the add tool
agent = ReActAgent.from_tools(functions, llm=llm, verbose=True)

# Executando o AI Agent

In [8]:
prompt =  """

Quanto é 1947279 multiplicado pelo quadrado de 19382912? Only make the calculations using the provided tools.

"""

result = agent.chat(prompt) #chat(str(messages))


> Running step 2812d90d-1e12-4f50-bafe-49d2cdf256ac. Step input: 

Quanto é 1947279 multiplicado pelo quadrado de 19382912? Only make the calculations using the provided tools.


Thought: The current language of the user is: Portuguese. I need to use a tool to help me answer the question.
Action: square
Action Input: {'inputs': '19382912'}
Observation: 375697277599744
> Running step db21363b-cbd7-4307-993d-244518c1b081. Step input: None
Thought: I need to use another tool to help me answer the question.
Action: multiply
Action Input: {'inputs': '1947279,375697277599744'}
Observation: 731587419027151896576
> Running step 904145aa-3336-47d4-af34-c9750e897cac. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: O resultado da multiplicação é **731587419027151896576**.
O resultado da multiplicação é **731587419027151896576**.


In [9]:
result.response

'O resultado da multiplicação é **731587419027151896576**.'

In [10]:
(19382912*19382912)

375697277599744

In [11]:
1947279 *375697277599744

731587419027151896576